# Long Range Loss validation

This is based on the theory, given how the model improve over longer range context length, the loss value approaches 1 over longer context lengths.

In [2]:
GPU_DEVICES="auto"
ENABLE_WANDB=True
WANDB_PREFIX="infctx-v5-LR-validation-loss"
DEEPSPEED_STRAT="deepspeed_stage_2"

print("ENABLE_WANDB:", ENABLE_WANDB)
print("GPU_DEVICES:", GPU_DEVICES)

if ENABLE_WANDB:
    WANDB_MODE="online"
else:
    WANDB_MODE="disabled"

# The model sizing
MODEL_NAME="RWKV-v5-Eagle-World-7B-v2-20240128-ctx4096.pth"
MODEL_URL="https://huggingface.co/RWKV/v5-Eagle-7B/resolve/main/RWKV-v5-Eagle-World-7B-v2-20240128-ctx4096.pth?download=true"

# Computing the notebook, and various paths
import os
NOTEBOOK_DIR=os.path.dirname(os.path.abspath("__file__"))
PROJECT_DIR=os.path.abspath(os.path.join(NOTEBOOK_DIR, "../../"))
TRAINER_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v5/"))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("TRAINER_DIR:", TRAINER_DIR)
print("PROJECT_DIR:", PROJECT_DIR)

ENABLE_WANDB: True
GPU_DEVICES: auto
NOTEBOOK_DIR: /workspace/picocreator/RWKV-infctx-trainer/notebook/trainer-v5-validation
TRAINER_DIR: /workspace/picocreator/RWKV-infctx-trainer/RWKV-v5
PROJECT_DIR: /workspace/picocreator/RWKV-infctx-trainer


# Preload the model and the data

In [3]:
# Lets wget the model files
!mkdir -p "{PROJECT_DIR}/model"
!cd "{PROJECT_DIR}/model" && \
    wget -O "{MODEL_NAME}" -nc "{MODEL_URL}"

File ‘RWKV-v5-Eagle-World-7B-v2-20240128-ctx4096.pth’ already there; not retrieving.


In [5]:
# Lets preload the requried dataset 
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/config/enwiki_100k-world-4096.yaml"

Map (num_proc=160): 100%|███| 1000000/1000000 [00:17<00:00, 57881.62 examples/s]
Filter (num_proc=160): 100%|█| 1000000/1000000 [00:06<00:00, 158000.06 examples/
Map (num_proc=160): 100%|███████| 79904/79904 [00:04<00:00, 19456.14 examples/s]
Saving the dataset (4/4 shards): 100%|█| 79904/79904 [00:04<00:00, 19732.78 exam
Saving the dataset (1/1 shards): 100%|█| 808/808 [00:00<00:00, 10472.99 examples


In [6]:
# Lets preload the requried dataset 
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/config/enwiki_100k-world-16k-rechunk.yaml"

Map (num_proc=160): 100%|███| 1000000/1000000 [00:17<00:00, 56798.77 examples/s]
Filter (num_proc=160): 100%|█| 1000000/1000000 [00:06<00:00, 166184.25 examples/
Map (num_proc=160): 100%|████████| 19916/19916 [00:03<00:00, 5006.21 examples/s]
Saving the dataset (4/4 shards): 100%|█| 19916/19916 [00:03<00:00, 5061.16 examp
Saving the dataset (1/1 shards): 100%|█| 202/202 [00:00<00:00, 2671.59 examples/


In [7]:
# Lets preload the requried dataset 
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/config/enwiki_100k-world-32k-rechunk.yaml"

Saving the dataset (4/4 shards): 100%|█| 9921/9921 [00:04<00:00, 2306.06 example
Saving the dataset (1/1 shards): 100%|█| 101/101 [00:00<00:00, 1322.34 examples/


# Validation loss for 4k / BS=1

In [17]:
# With microbatch = 1
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python3 lightning_trainer.py validate \
        -c "{NOTEBOOK_DIR}/config/enwiki_100k-world-4096.yaml" \
        --model.load_model="../model/{MODEL_NAME}" \
        --data.skip_datapath_setup=True \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - 7B - 4k loss validation (bs=1, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.microbatch_size=1 \
        --model.ctx_len=4096 \
        --trainer.devices="{GPU_DEVICES}"

[2024-02-04 12:42:57,250] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV infctx using 'torch-jit' with torch '2.1.2'
/home/picocreator/anaconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/pytorch/cli.py:518: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['validate', '-c', '/home/picocreator/rwkv-proj/RWKV-infctx-trainer/notebook/trainer-v5-validation/config/enwiki_100k-world-4096.yaml', '--model.load_model=../model/RWKV-v5-Eagle-World-7B-v2-20240128-ctx4096.pth', '--data.skip_datapath_setup=True', '--trainer.logger.init_args.name=infctx-v5-LR-validation-loss - 7B - 4k loss validation (bs=1, deepspeed_stage_2)', '--trainer.strategy=deepspeed_stage_2', '--trainer.microbatch_size=1', '--model.ctx_len=4096', '--trainer.devices=auto'], arg

# Validation loss for 16k / BS=1

In [16]:
# With microbatch = 1
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python3 lightning_trainer.py validate \
        -c "{NOTEBOOK_DIR}/config/enwiki_100k-world-16k-rechunk.yaml" \
        --model.load_model="../model/{MODEL_NAME}" \
        --data.skip_datapath_setup=True \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - 7B - 16k loss validation (bs=1, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.microbatch_size=1 \
        --model.ctx_len=4096 \
        --trainer.devices="{GPU_DEVICES}"

[2024-02-04 12:33:49,231] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV infctx using 'torch-jit' with torch '2.1.2'
/home/picocreator/anaconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/pytorch/cli.py:518: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['validate', '-c', '/home/picocreator/rwkv-proj/RWKV-infctx-trainer/notebook/trainer-v5-validation/config/enwiki_100k-world-16k-rechunk.yaml', '--model.load_model=../model/RWKV-v5-Eagle-World-7B-v2-20240128-ctx4096.pth', '--data.skip_datapath_setup=True', '--trainer.logger.init_args.name=infctx-v5-LR-validation-loss - 7B - 16k loss validation (bs=1, deepspeed_stage_2)', '--trainer.strategy=deepspeed_stage_2', '--trainer.microbatch_size=1', '--model.ctx_len=4096', '--trainer.devices=aut

# Validation loss for 4k / BS=4

In [15]:
# With microbatch = 4
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python3 lightning_trainer.py validate \
        -c "{NOTEBOOK_DIR}/config/enwiki_100k-world-4096.yaml" \
        --model.load_model="../model/{MODEL_NAME}" \
        --data.skip_datapath_setup=True \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - 7B - 4k loss validation (bs=4, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.microbatch_size=4 \
        --model.ctx_len=4096 \
        --trainer.devices="{GPU_DEVICES}"

[2024-02-04 12:26:32,718] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV infctx using 'torch-jit' with torch '2.1.2'
/home/picocreator/anaconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/pytorch/cli.py:518: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['validate', '-c', '/home/picocreator/rwkv-proj/RWKV-infctx-trainer/notebook/trainer-v5-validation/config/enwiki_100k-world-4096.yaml', '--model.load_model=../model/RWKV-v5-Eagle-World-7B-v2-20240128-ctx4096.pth', '--data.skip_datapath_setup=True', '--trainer.logger.init_args.name=infctx-v5-LR-validation-loss - 7B - 4k loss validation (bs=4, deepspeed_stage_2)', '--trainer.strategy=deepspeed_stage_2', '--trainer.microbatch_size=4', '--model.ctx_len=4096', '--trainer.devices=auto'], arg

# Validation loss for 16k / BS=4

In [14]:
# With microbatch = 1
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python3 lightning_trainer.py validate \
        -c "{NOTEBOOK_DIR}/config/enwiki_100k-world-16k-rechunk.yaml" \
        --model.load_model="../model/{MODEL_NAME}" \
        --data.skip_datapath_setup=True \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - 7B - 16k loss validation (bs=4, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.microbatch_size=4 \
        --model.ctx_len=4096 \
        --trainer.devices="{GPU_DEVICES}"

[2024-02-04 12:19:13,110] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV infctx using 'torch-jit' with torch '2.1.2'
/home/picocreator/anaconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/pytorch/cli.py:518: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['validate', '-c', '/home/picocreator/rwkv-proj/RWKV-infctx-trainer/notebook/trainer-v5-validation/config/enwiki_100k-world-16k-rechunk.yaml', '--model.load_model=../model/RWKV-v5-Eagle-World-7B-v2-20240128-ctx4096.pth', '--data.skip_datapath_setup=True', '--trainer.logger.init_args.name=infctx-v5-LR-validation-loss - 7B - 16k loss validation (bs=4, deepspeed_stage_2)', '--trainer.strategy=deepspeed_stage_2', '--trainer.microbatch_size=4', '--model.ctx_len=4096', '--trainer.devices=aut

In [8]:
# With microbatch = 8
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python3 lightning_trainer.py validate \
        -c "{NOTEBOOK_DIR}/config/enwiki_100k-world-32k-rechunk.yaml" \
        --model.load_model="../model/{MODEL_NAME}" \
        --data.skip_datapath_setup=True \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - 7B - 32k loss validation (bs=4, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.microbatch_size=8 \
        --model.ctx_len=4096 \
        --trainer.devices="{GPU_DEVICES}"

[2024-02-04 09:29:23,524] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV infctx using 'torch-jit' with torch '2.1.1+cu121'
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/cli.py:518: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['validate', '-c', '/workspace/picocreator/RWKV-infctx-trainer/notebook/trainer-v5-validation/config/enwiki_100k-world-32k-rechunk.yaml', '--model.load_model=../model/RWKV-v5-Eagle-World-7B-v2-20240128-ctx4096.pth', '--data.skip_datapath_setup=True', '--trainer.logger.init_args.name=infctx-v5-LR-validation-loss - 7B - 32k loss validation (bs=4, deepspeed_stage_2)', '--trainer.strategy=deepspeed_stage_2', '--trainer.microbatch_size=8', '--model.ctx_len=4096', '--trainer.devices=auto'], args=['validate', '-c', '/wo